## Figure 1: 

##### This notebook reproduces the analysis used to produce Figure 1, which incorporates the equations discussed in Section 2.1 of Maryann et al. 2025 for asteroid 2024 ON's distance measurement. All steps, parameters, and fits match the methodology described in Section 2.2 of the manuscript.

##### Step 1: Here we import all the packages used throughout the notebook:

In [51]:
# For numerical arrays and dataframes.
import numpy as np
import pandas as pd

# For handling plotting and axis formatting
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

# For custom subplot layouts
from matplotlib.gridspec import GridSpec

# Tools for fitting functions to data
from astropy.modeling import models, fitting
from scipy.optimize import curve_fit

# For file handling and regular expressions
import re
import os
from pathlib import Path

# Access to trigonometric functions and constants.
import math

##### Step 2: Reading and Extracting the raw data of asteroid 2024 ON from the NASA JPL Horizon System

In [52]:
# ------- Here, we specify the text file downloaded from Horizons, which is mentioned in GitHub folder 1 as 'Figure 1 (2024 ON).txt', containing only the selected columns needed for analysis. 
txt_files = ['Figure 1 (2024 ON).txt']

# In the DataFrame's list, the following are the column headers. The blanks denoted by '' correspond to columns that are not required in the analysis. 
# Date__(UT)__HR:MN = the time and day for each of the 48 observations that were taken during the time frame of September 5-6, 2024, separated by 1 hour, totaling 48 observations. 
# R.A._(ICRF), DEC__(ICRF) = The RA and DEC data of each observation in decimal format
# delta = the distance to the asteroid at each observation in astronomical units (au). We use the Horizons distance to verify the measured topocentric parallax distance in steps 14 and 15, thereby confirming the measured difference from the true distance. 
# deldot = the range rate (km/sec). We do not use this column in this code, but FYI, a positive (+) deldot indicates that an object is moving away from the observer, and a negative (-) deldot indicates that an object is moving towards the observer.
columns = ['Date__(UT)__HR:MN', '', '', ' R.A._(ICRF)', 'DEC__(ICRF)', 'delta', '     deldot', '']

# ------- We convert the document from a .txt file to a .csv file in the following steps, and then use pandas.DataFrame to be created in the new file, using the same column names as seen in the original file.

# Though the following loop can process multiple .txt files, since we are analyzing only a single asteroid, it will iterate over only a single .txt file. 
for file_path in txt_files:

    # We only want to extract the code between “SOE” (Start of Ephemeris) and “EOE” (End of Ephemeris), since they contain the ephemeris data. 
    start_marker = "$$SOE"
    end_marker = "$$EOE"
    
    # open: opens the file in read mode using 'r'. 
    # with: ensures that the file is closed properly after read.
    with open(file_path, 'r') as file:
        
        # data_lines: collects the data --> row by row between SOE and EOE.
        # is_data: A Boolean flag indicating if a row of data is under analysis and between SOE and EOE, then the data needs to be stored. 
        data_lines = []
        is_data = False 

        # Iterates through every line 
        for line in file:

            # When a row of lines contains SOE (the start marker), 'is_data' is set to True, and beyond this line, data will be captured.
            if start_marker in line:
                is_data = True
            # When a row of line contains EOE (the end marker), 'is_data' is set to False, and beyond this line, data will not be captured.
            elif end_marker in line:
                is_data = False
            # In the case that neither marker is found, and 'is_data' is set to true, it means that data capturing is still on-going, and hasn't reached EOE. 
            elif is_data:
                # .strip(): removes whitespaces and sends rows to data_lines.
                data_lines.append(line.strip())

    # Each line of ephemeris data that is captured is split at the commas to form string values so that they are in the form of rows and columns (lists of lists) for the .csv file. 
    data = [line.split(",") for line in data_lines]
    
    # max_columns: searches for the longest row and length.
    max_columns = max(len(row) for row in data)

    # Short rows are padded with empty strings at locations that do not have data, to represent the same length as the longest row. It ensures the DataFrame is created without misaligned columns.
    data = [row + [''] * (max_columns - len(row)) for row in data]

    # Converts the data in the rows and columns into a pandas.DataFrame.
    df = pd.DataFrame(data, columns=columns)

    # os.path.basename(file_path): extracts the filename
    # os.path.splitext(...)[0]: removes the .txt file extension so that a new .csv extension can be added
    base_name = os.path.splitext(os.path.basename(file_path))[0]
    csv_file_path = f'{base_name}.csv'
    df.to_csv(csv_file_path, index=False)
    print(f"CSV file saved: {csv_file_path}")

CSV file saved: Figure 1 (2024 ON).csv


##### Step 3: Reading of the newly created .csv file. 

In [53]:
# Here, we read and print the newly created .csv file to verify if the data is correctly extracted and formatted from the .txt file. 
file_path1 = 'Figure 1 (2024 ON).csv'
data1 = pd.read_csv(file_path1)
print(data1.to_string(index=False))

 Date__(UT)__HR:MN Unnamed: 1 Unnamed: 2   R.A._(ICRF)  DEC__(ICRF)    delta       deldot  Unnamed: 7
      2.460558e+06                     m    269.097322     5.569631 0.063274    -8.567432         NaN
      2.460559e+06                          269.093467     5.550391 0.063069    -8.465509         NaN
      2.460559e+06                          269.090382     5.530812 0.062867    -8.372713         NaN
      2.460559e+06                          269.088615     5.510917 0.062666    -8.295454         NaN
      2.460559e+06                          269.088634     5.490742 0.062467    -8.239074         NaN
      2.460559e+06                          269.090788     5.470335 0.062269    -8.207475         NaN
      2.460559e+06                          269.095285     5.449750 0.062072    -8.202861         NaN
      2.460559e+06                          269.102180     5.429042 0.061874    -8.225582         NaN
      2.460559e+06                          269.111365     5.408268 0.061676    -8

##### Step 4: Addition of a randomly generated 20 milliarcseconds (mas) Gaussian noise to each observation in the RA and DEC. 

In [54]:
# input_file: The original file path with RA and DEC.
# output_file: The new file path with noise added in the RA and DEC. 
input_file  = Path("Figure 1 (2024 ON).csv")
output_file = Path("Figure 1 (2024 ON)_20mas.csv")

# The columns that need noise addition in the original file. 
RA_COL  = " R.A._(ICRF)"      
DEC_COL = "DEC__(ICRF)"

# Horizons does not provide measurement uncertainties in RA and DEC. So, we randomly add 20 mas of noise to mimic real astrometric errors.
# 20 mas converted to arcseconds. 1 mas = 0.001 arcsec and 20 mas = 0.020 arcsec
# The noise is Gaussian-distributed, consistent with typical astrometric uncertainties.
# 20 mas was chosen since it is a realistic uncertainty approximation based on typical CCD astrometric performance.
SIGMA_ARCSEC = 0.020

# converting arcseconds into degrees because RA/DEC in the original Horizons file is in degrees (1 degree = 3600"). 
SIGMA_DEG    = SIGMA_ARCSEC / 3600.0   

# A random generator. seed=42 ensures reproducible noise, where every run of the code ensures the noise in the observations stays the same. 
rng = np.random.default_rng(seed=42)

# Reads the .csv into a DataFrame, where each row is an observation.
df = pd.read_csv(input_file)

# A safety check: If RA or DEC columns are missing, stop the code and show an error.
if RA_COL not in df.columns or DEC_COL not in df.columns:
    raise ValueError("RA/DEC columns not found in file")

# Extract the RA and DEC columns into a NumPy array.
ra  = df[RA_COL].to_numpy(dtype=float)
dec = df[DEC_COL].to_numpy(dtype=float)

# Generate one random noise per RA and DEC observation, where mean = 0, standard deviation = 20 mas in degrees, size = number of observations (48).
ra_noise  = rng.normal(0.0, SIGMA_DEG, size=len(df))
dec_noise = rng.normal(0.0, SIGMA_DEG, size=len(df))

# Adds noise to each RA. 
# % 360.0: wraps RA so that values remain within 0–360 degrees. 
ra_new  = (ra + ra_noise) % 360.0

# Adds noise to each DEC. 
# Limits the DEC at -90 and +90 since it cannot exceed the poles. 
dec_new = np.clip(dec + dec_noise, -90.0, 90.0)

# Printing a readable header to see the comparison. 
print(f"\n=== {input_file.name} ===")
print("Obs |     Original_RA     Original_DEC |    RA_noise(deg)   DEC_noise(deg) |   RA_noise(mas)   DEC_noise(mas) |          New_RA          New_DEC")
print("----+------------------+----------------+----------------------------------+----------------------------------+---------------------------------")
for i in range(len(df)):
    print(f"{i+1:3d} | "
          f"{ra[i]:16.9f} {dec[i]:15.9f} | "
          f"{ra_noise[i]:16.9f} {dec_noise[i]:16.9f} | "
          f"{ra_noise[i]*3600*1000:16.9f} {dec_noise[i]*3600*1000:15.9f} | "
          f"{ra_new[i]:16.13f} {dec_new[i]:16.13f}")

# In the newly created .csv file, the original Horizons .csv file is now replaced with 20mas noise in the RA and DEC. 
df[RA_COL]  = ra_new
df[DEC_COL] = dec_new
df.to_csv(output_file, index=False)
print(f"Saved {output_file}")


=== Figure 1 (2024 ON).csv ===
Obs |     Original_RA     Original_DEC |    RA_noise(deg)   DEC_noise(deg) |   RA_noise(mas)   DEC_noise(mas) |          New_RA          New_DEC
----+------------------+----------------+----------------------------------+----------------------------------+---------------------------------
  1 |    269.097322467     5.569630723 |      0.000001693      0.000003772 |      6.094341595    13.578271261 | 269.0973241598727  5.5696344947420
  2 |    269.093467356     5.550391266 |     -0.000005778      0.000000375 |    -20.799682125     1.351581390 | 269.0934615783106  5.5503916414393
  3 |    269.090381735     5.530811880 |      0.000004169      0.000001606 |     15.009023916     5.782387974 | 269.0903859041733  5.5308134862189
  4 |    269.088615243     5.510916574 |      0.000005225      0.000003507 |     18.811294328    12.625764517 | 269.0886204683595  5.5109200811568
  5 |    269.088633890     5.490742097 |     -0.000010839     -0.000008095 |    -39.020703

##### Step 5: Reading of the newly created .csv file with noise. 

In [55]:
# We print the noise-filled RA and DEC to verify if the data is correctly extracted and formatted. 
file_path = 'Figure 1 (2024 ON)_20mas.csv'
data = pd.read_csv(file_path)
print(data.to_string(index=False))

 Date__(UT)__HR:MN Unnamed: 1 Unnamed: 2   R.A._(ICRF)  DEC__(ICRF)    delta       deldot  Unnamed: 7
      2.460558e+06                     m    269.097324     5.569634 0.063274    -8.567432         NaN
      2.460559e+06                          269.093462     5.550392 0.063069    -8.465509         NaN
      2.460559e+06                          269.090386     5.530813 0.062867    -8.372713         NaN
      2.460559e+06                          269.088620     5.510920 0.062666    -8.295454         NaN
      2.460559e+06                          269.088623     5.490734 0.062467    -8.239074         NaN
      2.460559e+06                          269.090780     5.470334 0.062269    -8.207475         NaN
      2.460559e+06                          269.095286     5.449747 0.062072    -8.202861         NaN
      2.460559e+06                          269.102178     5.429038 0.061874    -8.225582         NaN
      2.460559e+06                          269.111365     5.408266 0.061676    -8

##### Step 6: Printing only the time and RA needed for the topocentric parallax distance measurement analysis. 

In [56]:
# Subtracting the minimum in the time column to make the first observation start at 0
# By shifting the first observation to x = 0, we improve numerical stability in nonlinear least-squares fitting

jd_adjusted = data['Date__(UT)__HR:MN']
jd_binned_median = jd_adjusted - jd_adjusted.min()

# Extracting the RA values
ra_deg_adjusted_detected = data[' R.A._(ICRF)']

print(f"{'JD Adjusted':<25}{'RA Adjusted Detected':<25}")
print("=" * 75)

for jd, ra in zip(jd_binned_median, ra_deg_adjusted_detected):
    print(f"{jd:<25.10f}{ra:<25.10f}")

JD Adjusted              RA Adjusted Detected     
0.0000000000             269.0973241599           
0.0416666670             269.0934615783           
0.0833333330             269.0903859042           
0.1250000000             269.0886204684           
0.1666666670             269.0886230509           
0.2083333330             269.0907802807           
0.2500000000             269.0952858562           
0.2916666670             269.1021782321           
0.3333333330             269.1113649687           
0.3750000000             269.1225750409           
0.4166666670             269.1354319215           
0.4583333330             269.1494039471           
0.5000000000             269.1639146158           
0.5416666670             269.1783570935           
0.5833333330             269.1920971793           
0.6250000000             269.2045732002           
0.6666666670             269.2153219126           
0.7083333330             269.2239537649           
0.7500000000             269.23

##### Step 7: Equation 5 in Maryann et al. 2025 (Quadratic function)

In [57]:
# Equation 5 in the paper. We use the quadratic function since the observations for 2024 ON in this analysis exceed a single night. 
# Am * np.sin(2 * np.pi * x + B) is the Earth's rotation (x = time; Am = amplitude of the signal, also known as the parallax amplitude; B = phase shift). 
# C * x + D + E * x**2 is the asteroids non-linear motion (Cx = asteroids linear motion showing a constant angular velocity; D = RA offset; E x^2 = asteroids non-linear motion due to changing geometry and orbital acceleration)
# fit_function_linear defines a Python function

def fit_function_linear(x, A, B, C, D, E):
    return A * np.sin(2 * np.pi * x + B) + C * x + D + E * x**2

# curve_fit() from scipy.optimize --> performs the nonlinear least-squares fit. It finds the best fit value for A, B, C, D, and E that fit into the RA data with the 20mas noise.
# params_sine = a NumPy array of best-fit parameter values
# covariance_sine = 5×5 covariance matrix describing the uncertainties and correlations for A, B, C, D, and E.

params_sine, covariance_sine = curve_fit(fit_function_linear, jd_binned_median, ra_deg_adjusted_detected)
print(f"\033[1;32m Equation 5: sine +linear terms (Parameters):     {params_sine}\033[0m")

# Uncertainties in Equation 5. 
# It is important for understanding the error propagation. 
# np.diag(covariance_sine): extracts diagonal elements and the variances of A, B, C, D, and E.
# np.sqrt(...): takes the square root and gives the standard deviations (1σ uncertainties).

sine_uncertainties = np.sqrt(np.diag(covariance_sine))
print(f"\033[1;32m Equation 5: sine +linear terms (Uncertainities):{sine_uncertainties}\033[0m")

 Equation 5: sine +linear terms (Parameters):     [-3.64927760e-02  9.21010340e-02  1.05446742e-01  2.69102671e+02
  2.22951403e-02]
 Equation 5: sine +linear terms (Uncertainities):[0.00017452 0.0045342  0.00079852 0.00034224 0.00039287]


##### Step 8: The parallax amplitude and uncertainty from the asteroids non-linear sinusoidal signal

In [58]:
# We read only the parallax amplitude and its uncertainty, and convert to radians. 
# A closer asteroid → larger sinusoidal amplitude
# A farther asteroid → smaller sinusoidal amplitude

amplitude_deg = params_sine[0]
amplitude_rad = math.radians(amplitude_deg)
print(f"\033[1;32mamplitude (rad):              {amplitude_rad}\033[0m")
 
amplitude_uncertainty_deg = sine_uncertainties[0]
amplitude_uncertainty_rad = math.radians(amplitude_uncertainty_deg)
print(f"\033[1;32mamplitude uncertianity (rad): {amplitude_uncertainty_rad}\033[0m")

amplitude (rad):              -0.0006369190946174064
amplitude uncertianity (rad): 3.0459485392267557e-06


##### Step 9: Equation 4 in Maryann et al. 2025 (Distance measurement with respect to the center of the Earth)

In [59]:
earth_radius_km = 6371  # in km

# Distance measurement concerning the center of the Earth. 
# Since the measurements do not have an observatory's latitude or the DEC of the observations, topocentric parallax is not accounted for in the calculations. To fix this, we need two adjustments as seen in steps 10 and 11: 
distance_km = earth_radius_km / amplitude_rad

# Compute distance uncertainty
distance_uncertainty_km = (earth_radius_km/amplitude_rad) * (amplitude_uncertainty_rad/abs(amplitude_rad))

print(f"\033[1;32mEquation 3: Distance_km: {abs(distance_km):.2f} ± {abs(distance_uncertainty_km):.2f}\033[0m")

Equation 3: Distance_km: 10002840.32 ± 47836.75


##### Step 10: Latitude of the Observatory (Correction 1)

##### Note: The Cerro Tololo latitude used in the published manuscript (30.1732° S and altitude of 2200 m) differs slightly from the precise geodetic latitude reported by JPL Horizons (−30.1691165°). The code released here uses the Horizons value. This difference has a negligible effect on the topocentric parallax calculation and does not affect any results or conclusions in the paper.

In [60]:
# The observatory taking the observations of asteroid 2024 ON is the Cerro Tololo Observatory, La Serena, (code: 807) 
# The observatory is situated at an altitude of 2388 m
# The observatory is in the southern hemisphere of the Earth at a latitude of 30.169° S. 
# We convert the latitude from degrees to radians. 
# Taken from the Horizons file: Center geodetic: 289.1941, -30.1691165, 2.38888  {E-lon(deg), Lat(deg), Alt(km)}

latitude_deg = 30.1691165
latitude_rad = math.radians(latitude_deg)
print("Latitude (rad):              ", latitude_rad)

Latitude (rad):               0.5265504153427478


##### Step 11: Declination (DEC) of the observations (Correction 2)

In [61]:
# Since the DEC of the observations is over two days, the change in the degrees is very small, and so we assume a single DEC value fixed at the first observation.

dec_deg = data['DEC__(ICRF)'][0]
dec_rad = math.radians(dec_deg)
print("DEC (rad):                   ", dec_rad)

DEC (rad):                    0.09720846006589902


##### Step 12: Equation 3 in Maryann et al. 2025

In [62]:
# We correct for the distance measurement in step 9 with the latitude and DEC as discussed in steps 11 and 12. 
asteroid_km = (distance_km * math.cos(latitude_rad)) / math.cos(dec_rad)

# Uncertainty
asteroid_uncertainty_km = asteroid_km * (distance_uncertainty_km / distance_km)

print(f"\033[1;32mEquation 4: Asteroid Distance (km):{abs(asteroid_km):.2f} ± {abs(asteroid_uncertainty_km):.2f}\033[0m")

Equation 4: Asteroid Distance (km):8688934.38 ± 41553.23


##### Step 13: Asteroid 2024 ON distance converted to astronomical units. 

In [63]:
km_to_au = 149597870.7 # from km to au 

# distance converted to au 
asteroid_dist_au = asteroid_km / km_to_au

# distance uncertainty converted to au
asteroid_dist_uncertainty_au = asteroid_uncertainty_km / km_to_au

print(f"\033[1;32mAsteroid Distance (au):{abs(asteroid_dist_au):.6f} ± {abs(asteroid_dist_uncertainty_au):.6f}\033[0m")

Asteroid Distance (au):0.058082 ± 0.000278


##### Step 14: Extracting Mean Distance from Horizons 

In [64]:
delta = data['delta']
true_distance = delta.mean()
print(f"\033[1;32mNASA JPL Horizons True distance (au): {true_distance}\033[0m")

NASA JPL Horizons True distance (au): 0.05844223491561792


##### Step 15: Comparing measured asteroid distance with Horizons distance which is converted to the mean distance in step 14.

In [65]:
if asteroid_dist_au == 0 or true_distance == 0:
    print("\033[1;31mError: One of the values is zero, cannot compute percentage difference.\033[0m")
else:
    percentage_difference = abs(abs(asteroid_dist_au) - abs(true_distance)) / ((abs(asteroid_dist_au) + abs(true_distance)) / 2) * 100
    print(f"\033[1;32mAsteroid Distance Difference between True and Predicted: {percentage_difference:.6f}%\033[0m")

Asteroid Distance Difference between True and Predicted: 0.618406%


##### Step 16: Retesting the results as per the exact equations seen in 3 and 4 in the manuscript. 

In [ ]:
Dec      = dec_rad
latitude = latitude_rad
Am       = amplitude_rad
Am_sigma = amplitude_uncertainty_rad

# Uncertainties in the final distance measurement is error propogated through a standard first-order Gaussian.

# ---------- Eq. 3: --------------------------------
# The Projected Parallax amplitude and uncertainty
# If Ap is large, the distance is well-constained and the object is closer to the observer, and vice versa

Ap               = (math.cos(Dec)/math.cos(latitude))*Am
Ap_uncertainties = (math.cos(Dec)/math.cos(latitude))*Am_sigma
print(Ap)
print(Ap_uncertainties)
print(f"\033[1;32mAp: {abs(Ap):.6f} ± {abs(Ap_uncertainties):.6f}\033[0m")

# ---------- Eq. 4: --------------------------------
# The asteroids distance is inverse proportional to the parallax amplitude
Asteroid_Distance_km               = earth_radius_km                     /Ap

# and the uncertainty is   ==>   σ_D = |∂D/∂Ap| * σ_Ap    ==>    σ_D = R_earth * σ_Ap / Ap^2 
Asteroid_Distance_km_uncertainties = earth_radius_km * Ap_uncertainties / (Ap**2) 

print(Asteroid_Distance_km)
print(Asteroid_Distance_km_uncertainties)
print(f"\033[1;32mAsteroid Distance (km): {abs(Asteroid_Distance_km):.6f} ± {abs(Asteroid_Distance_km_uncertainties):.6f}\033[0m")

# ---------- Distance (au) ------------------------
Asteroid_Distance_au               = Asteroid_Distance_km              /km_to_au
Asteroid_Distance_au_uncertainties = Asteroid_Distance_km_uncertainties/km_to_au
print(Asteroid_Distance_au)
print(Asteroid_Distance_au_uncertainties)
print(f"\033[1;32mAsteroid Distance (km): {abs(Asteroid_Distance_au):.6f} ± {abs(Asteroid_Distance_au_uncertainties):.6f}\033[0m")

# ---------- % difference -------------------------
if asteroid_dist_au == 0 or true_distance == 0:
    print("\033[1;31mError: One of the values is zero, cannot compute percentage difference.\033[0m")
else:
    percentage_difference = abs(abs(Asteroid_Distance_au) - abs(true_distance)) / ((abs(Asteroid_Distance_au) + abs(true_distance)) / 2) * 100
    print(f"\033[1;32mHorizons versus Measured: {percentage_difference:.6f}%\033[0m")

##### Step 17: Plotting the motion of asteroid 2024 ON. We also highlight the region in the observations where real-time observations of 2024 ON was taken on the night of September 5-6, 2025, from the PROMPT telescope in Chile, as discussed in Figures 5 and 6 in Maryann et al. 2025. 

In [ ]:
radians_to_degrees = 180 / np.pi

fig = plt.figure(figsize=(20, 12))
gs = GridSpec(3, 1, height_ratios=[1.5, 1.5, 1], figure=fig)

# Subplot 1: 
ax1 = fig.add_subplot(gs[0, 0])
ax1.scatter(jd_binned_median, ra_deg_adjusted_detected, color='blue', label = '2024 ON synthetic data')
ax1.plot(jd_binned_median, fit_function_linear(jd_binned_median, *params_sine), color='green', label = 'Equation 5 fit')
ax1.axvspan(0.988, 1.144, color='red', alpha=0.2, label="Time window of 2024 ON's real-time \n observations, as shown in Figure 6")
ax1.tick_params(axis='both', which='major', labelsize=16)
ax1.set_ylabel("RA (degrees)", fontsize=16)
ax1.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
ax1.set_xticklabels([])
ax1.legend(fontsize=16)
ax1.grid()

# Subplot 2: 
ax2 = fig.add_subplot(gs[1, 0])
ax2.plot(jd_binned_median, 
         fit_function_linear(jd_binned_median, *params_sine) - (params_sine[2]*jd_binned_median + params_sine[3] + params_sine[4]*jd_binned_median**2), 
         color = 'green', label = 'Asteroid\'s motion subtracted from Equation 5')
ax2.scatter(jd_binned_median, ra_deg_adjusted_detected - (params_sine[2]*jd_binned_median + params_sine[3] + params_sine[4]*jd_binned_median**2), 
            color = 'blue', label = 'Asteroid\'s motion subtracted from data')
ax2.axvspan(0.988, 1.144, color='red', alpha=0.2)
ax2.tick_params(axis='both', which='major', labelsize=16)
ax2.set_xlabel("Normalized Julian Date", fontsize=16)
ax2.set_ylabel("Residuals (degrees)", fontsize=16) 
ax2.yaxis.set_major_formatter(FormatStrFormatter('%.3f'))
ax2.legend(fontsize=16)
ax2.grid()

plt.subplots_adjust(hspace=0)
plt.gcf().axes[0].spines['bottom'].set_linewidth(2)
plt.gcf().axes[1].spines['top'].set_linewidth(2)
plt.subplots_adjust(left=0.1, right=0.95, top=0.95, bottom=0.1, wspace=0.1, hspace=0)
plt.savefig("Figure 1.png", format="png", dpi=300, bbox_inches="tight", facecolor='white')
plt.show()

##### FINAL SUMMARY: Topocentric Parallax distance measurement for Asteroid 2024 ON

In [ ]:
# 1. Fit parameters from Equation 5
print("=== Fit Parameters from Equation 5 ===")
parameter_names = ["Am (deg)", "B", "C", "D", "E"]
for name, value, error in zip(parameter_names, params_sine, sine_uncertainties):
    print(f"{name:<10}: {value:.6e} ± {error:.6e}")

# 2. Parallax amplitude in radians
print("\n=== Parallax Amplitude ===")
print(f"Am (rad): {amplitude_rad:.6e} ± {amplitude_uncertainty_rad:.6e}")

# 3. Distance measurement
print("\n=== Distance to Asteroid 2024 ON ===")
print(f"Distance:    {abs(asteroid_km):.2f} km   ({abs(asteroid_dist_au):.6f} au)")
print(f"Uncertainty: ± {abs(asteroid_uncertainty_km):.2f} km   (± {abs(asteroid_dist_uncertainty_au):.6f} au)")

# 4. Percentage Difference
print("\n=== Difference comparism with Horizons mean distance ===")
print(f"Percentage: {abs(percentage_difference):.2f} %")


# 4. Summary Text
summary_text = """ 
Summary:
--------
We input the data of asteroid 2024 ON from NASA JPL Horizons System from September 5-6, 2024, 
separated by 1 hour, totaling 48 observations. We add synthetic Gaussian noise of σ = 20 mas 
to the RA and DEC of the original data, to mimic realistic astrometric scatter, since 
Horizons does not provide measurement uncertainties. We then fit Equation 5 in the manuscript, 
which connects the Earth's rotation with the non-linear motion of the asteroid to 
the RA time series over the two days. 

The best-fit sinusoidal amplitude (Am) and its 1σ uncertainty were converted 
to radians and used in Equations 3 and 4 of the manuscript to compute the 
distance to asteroid 2024 ON. The resulting distance and uncertainties are consistent 
with expectations for an object at this geometry and validate that the topocentric parallax 
method is implemented correctly. This confirms that the entire analysis pipeline: 
noise generation → model fitting → amplitude extraction → distance calculation, 
is functioning as described in the manuscript.
"""
print("\n" + summary_text)
